In [2]:
search_for = 137
start_from = 0
threads = 10
thread = 3
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.004971027374267578
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 354865621
type: [1, 1, 1, 1, 137] 137
cases of this type: 352275361
100000 19.893958011061113
200000 33.36343447036912
300000 33.18130454347549
400000 32.50644668502225
500000 32.28857501910134
600000 32.60493081426288
700000 33.835821330025404
800000 32.49524305645266
900000 32.09381162001909
1000000 31.892071063782616
1100000 31.61586358264167
1200000 31.84216079068236
1300000 15.59296642421522
1400000 23.88489779101141
1500000 31.66946129625394
1600000 31.460899172590267
1700000 31.54422654664137
1800000 31.795772318453164
1900000 31.860156779128253
2000000 33.2328030335043
2100000 31.76398453200487
2200000 31.464501579818087
2300000 32.07724255596026
2400000 32.28902418321836
2500000 32.0640636172119
2600000 12.718040369857496
2700000 27.819458501353104
2800000 39.92077788995958
2900000 41.80249411000471
3000000 40.1097496878

29700000 29.85059581491387
29800000 32.31133062311604
29900000 32.87097699785116
30000000 33.303510529095036
30100000 33.31481711791596
30200000 32.891674362439204
30300000 33.35945973402275
30400000 35.393774712522195
30500000 33.71851664706016
30600000 32.835393133883045
30700000 12.584017036074497
30800000 35.658944426940934
30900000 34.85062892571059
31000000 34.0680581424112
31100000 20.809787149209782
31200000 24.52585957365401
31300000 33.8039740146629
31400000 34.199919147002
31500000 34.07383782397105
31600000 33.00189901169965
31700000 34.29553710941421
31800000 33.11544665398145
31900000 32.15212468529706
32000000 32.05875011171565
32100000 31.223238940037824
32200000 7.662819953820489
32300000 32.014962827656575
32400000 31.913021109691073
32500000 32.297725659244115
32600000 32.804258788908584
32700000 33.110875803787486
32800000 31.8370572779077
32900000 33.256457453156926
33000000 33.31245696502988
33100000 32.71768877564435
33200000 20.7359086830993
33300000 25.97048932

59600000 16.32378179920131
59700000 24.316206390723664
59800000 30.122282860068783
59900000 28.36986939096414
60000000 29.475410338419938
60100000 28.064691937960887
60200000 27.466024264416262
60300000 27.60169069972272
60400000 23.235302717831164
60500000 11.872241775893626
60600000 27.120813198057768
60700000 27.476356029605842
60800000 27.51746102515715
60900000 28.552418757603522
61000000 28.516673803653713
61100000 28.820109748876487
61200000 29.006232430943083
61300000 13.880740565001487
61400000 27.926863544174502
61500000 30.463314402700014
61600000 29.36706829393804
61700000 31.060514054391998
61800000 29.884262497271102
61900000 29.747515121761445
62000000 30.0296778314982
62100000 30.403102407510186
62200000 13.806765855539304
62300000 25.788388989956378
62400000 28.795198158214642
62500000 28.3701464891311
62600000 28.724302684314722
62700000 27.533401321805854
62800000 26.589357995059217
62900000 26.173606812015663
63000000 16.032387757091193
63100000 18.269142960540123
6

89300000 24.274494728716952
89400000 8.552239890742227
89500000 26.25156973900719
89600000 25.202276643811135
89700000 25.770358422941612
89800000 26.201497692432376
89900000 26.115177643620722
90000000 27.064263711894466
90100000 25.647192142398207
90200000 26.62204451167967
90300000 26.052736814935884
90400000 25.8749334469582
90500000 25.15746569900203
90600000 25.530095493965014
90700000 9.763407168324688
90800000 22.43740959459975
90900000 24.129769359905165
91000000 22.92745037435536
91100000 23.63500205847575
91200000 23.852045902002413
91300000 11.178274310710014
91400000 18.46791723757312
91500000 23.900143938083975
91600000 23.47867213388065
91700000 23.247130453472593
91800000 23.71501781007372
91900000 16.571816405132534
92000000 18.117399585339612
92100000 24.81594632476946
92200000 25.27428913971983
92300000 26.39336597747369
92400000 25.75183781754951
92500000 26.64410982896934
92600000 25.59421243552015
92700000 25.841968012493624
92800000 25.91032205410766
92900000 25.

118500000 23.921603694269884
118600000 23.809092166126423
118700000 24.223143935382197
118800000 22.36742326450979
118900000 22.176630342314592
119000000 21.69116716699418
119100000 21.600607503784513
119200000 6.847692891677767
119300000 22.19613585122613
119400000 21.994828033646623
119500000 21.283944938243117
119600000 7.484025689375549
119700000 20.084592114042294
119800000 21.021642404013942
119900000 22.19410249590045
120000000 11.795808716232122
120100000 18.36928360220611
120200000 22.002650445882708
120300000 21.377121369781403
120400000 21.682122670399668
120500000 22.89875368310055
120600000 22.84806070270682
120700000 23.90347882431746
120800000 23.53372521914843
120900000 23.71445842474576
121000000 24.320232159278756
121100000 23.51775460579353
121200000 23.2022498971194
121300000 23.239126323341996
121400000 22.19686637292964
121500000 21.942563378476798
121600000 21.370958312952137
121700000 22.10408297172819
121800000 6.614096996681774
121900000 21.16668606091374
1220

...

233100000 9.052455656073407
233200000 5.713964535734848
233300000 11.896988748828052
233400000 11.700319389034458
233500000 11.527355647515554
233600000 12.268572671686822
233700000 11.634980034890845
233800000 12.926717903732444
233900000 12.804846175848656
234000000 12.19086982198372
234100000 12.694768017546362
234200000 12.172091235127677
234300000 12.636169750442175
234400000 12.567326967224448
234500000 10.941894104589975
234600000 11.912561686571939
234700000 10.673358176931051
234800000 11.814461408550912
234900000 4.515835936898423
235000000 13.411063307361372
235100000 10.9562054280453
235200000 11.531286890487413
235300000 4.991372531342542
235400000 11.187938233406218
235500000 10.778241383892135
235600000 11.34264452760425
235700000 8.852469031661109
235800000 8.264808365684662
235900000 10.491431561699047
236000000 11.607564282489589
236100000 11.73814289810765
236200000 11.791822024676497
236300000 12.446011819700539
236400000 11.906164149606973
236500000 12.627486027338

262000000 9.901178183313409
262100000 10.004644001021093
262200000 9.809713558585512
262300000 9.445347913399535
262400000 9.518212594388627
262500000 9.594665877248282
262600000 9.68466164338387
262700000 9.067236220447286
262800000 9.394532675236883
262900000 7.6535782412720685
263000000 5.05042637909345
263100000 8.21199236027255
263200000 8.898089153210163
263300000 8.168871132543742
263400000 8.718885246752443
263500000 8.8839313899179
263600000 2.489608838072803
263700000 8.031114633808214
263800000 8.993837179290948
263900000 8.504740917825083
264000000 8.783878280672834
264100000 8.877168179566878
264200000 5.721382474361742
264300000 5.815764782802119
264400000 9.374432171369499
264500000 9.510117628533166
264600000 9.034263689146622
264700000 9.734766067426836
264800000 9.7933563189488
264900000 9.840510899966548
265000000 9.972706825982588
265100000 9.115963681601047
265200000 9.781686288734143
265300000 9.64363699351146
265400000 8.75298152029186
265500000 5.14188336083055


291300000 6.249287144598044
291400000 6.421106859239316
291500000 5.94749041223417
291600000 6.079369678985918
291700000 5.929260881321866
291800000 6.114102560247951
291900000 1.1471555180888215
292000000 6.209274765871024
292100000 5.923590388729534
292200000 6.048958488058733
292300000 5.940061858456966
292400000 6.265595350130085
292500000 6.278984533249884
292600000 6.29397584008247
292700000 4.331873512436999
292800000 4.576100118452826
292900000 6.715301561646902
293000000 6.147804121164614
293100000 6.756061774053697
293200000 6.691580888447323
293300000 6.264677478411122
293400000 6.652556259708492
293500000 6.438953801601549
293600000 2.353379305252134
293700000 6.602705963566708
293800000 6.3635856328566085
293900000 6.169257073963044
294000000 5.866487540757386
294100000 6.01346088582338
294200000 5.552245407677765
294300000 6.027752556247895
294400000 4.3440945351816875
294500000 3.1489063135016857
294600000 5.4474692756064655
294700000 5.917306261054189
294800000 5.531765

320400000 3.2621173303318036
320500000 3.338845055142862
320600000 3.3639977889246833
320700000 3.532134192073936
320800000 3.327974796022338
320900000 3.4498884709180593
321000000 3.460999904165775
321100000 3.4525025181154865
321200000 1.8852717740854383
321300000 2.742135993994809
321400000 3.5740118720146454
321500000 3.3969541947195974
321600000 3.189056100231867
321700000 1.479847504760444
321800000 3.3546098298351996
321900000 3.3789442424569365
322000000 3.5257264195193985
322100000 3.2344872035258554
322200000 3.1672611713612984
322300000 3.3238769318068346
322400000 3.2364823266957745
322500000 3.19564074657675
322600000 3.067842366761434
322700000 2.0344680919910885
322800000 1.882506105082565
322900000 3.0834042254324774
323000000 3.3200666608372957
323100000 3.1292998579787654
323200000 3.188391180734082
323300000 3.1033008358660923
323400000 3.1295676992405848
323500000 3.234435275074035
323600000 3.179681249186034
323700000 3.155244244524587
323800000 1.52464690835463
32

349000000 0.4947715866082489
349100000 0.44437693699410163
349200000 0.573323749793991
349300000 0.46753191354866225
349400000 0.4719567239433202
349500000 0.5428411954738537
349600000 0.5587568465298594
349700000 0.2358316520451612
349800000 0.21720736712377994
349900000 0.3752399223647336
350000000 0.3669306571818901
350100000 0.39684680095075237
350200000 0.30228342406446734
350300000 0.24767675375478135
350400000 0.22328033429372973
350500000 0.25977816434732676
350600000 0.31831199495566354
350700000 0.2870528523173882
350800000 0.2638494848954797
350900000 0.17850685088717277
351000000 0.1273230604743527
351100000 0.18745904726314744
351200000 0.23796838974743617
351300000 0.2030411876325594
351400000 0.17095716543751957
351500000 0.24523056682271424
351600000 0.2321706065523677
351700000 0.1944757334411303
351800000 0.1892821209819575
351900000 0.14542905819688703
352000000 0.1325675368019879
352100000 0.15770846820854015
352200000 0.17075260189902186
best so far: 0
type: [1, 1,